In [2]:
"""Import libraries and read in all .root files for the sm and eft samples"""

import numpy as np
import uproot
import vector
import awkward as ak
import math

# Open the .root file and extract the tau data

sm_events = []
eft_events = []

# Change commented out code if testing with different data

# for i in range(1,6):
#     sm_tree = uproot.open(f'../../Final_Samples/delphes_ee_zh_mumutautau_sm_PDFs_50000events_230224_Run{i}.root:Delphes')
#     sm_events.append(sm_tree.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 'Muon.Charge',
#                      'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.TauTag', 'Jet.Charge', 'Jet.Mass',
#                      'MissingET.MET', 'MissingET.Eta', 'MissingET.Phi']))
#     eft_tree = uproot.open(f'../../Final_Samples/delphes_ee_mumutautau_smeft_PDFs_50000events_040324_Run{i}.root:Delphes')
#     eft_events.append(eft_tree.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 'Muon.Charge',
#                      'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.TauTag', 'Jet.Charge', 'Jet.Mass',
#                      'MissingET.MET', 'MissingET.Eta', 'MissingET.Phi']))

# for i in range(1,6):
#     sm_tree = uproot.open(f'../../Final_Samples/delphes_ee_zh_mumutautau_sm_PDFs_50000events_230224_Run{i}.root:Delphes')
#     sm_events.append(sm_tree.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 'Muon.Charge',
#                      'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.TauTag', 'Jet.Charge', 'Jet.Mass',
#                      'MissingET.MET', 'MissingET.Eta', 'MissingET.Phi']))
#     eft_tree = uproot.open(f'../../Final_Samples/120324_chbtil_10_Run{i}.root:Delphes')
#     eft_events.append(eft_tree.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 'Muon.Charge',
#                      'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.TauTag', 'Jet.Charge', 'Jet.Mass',
#                      'MissingET.MET', 'MissingET.Eta', 'MissingET.Phi']))

# eft_tree = uproot.open(f'../../Final_Samples/130324_chbtil_0.1_Run1.root:Delphes')
# eft_events.append(eft_tree.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 'Muon.Charge',
#                  'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.TauTag', 'Jet.Charge', 'Jet.Mass',
#                  'MissingET.MET', 'MissingET.Eta', 'MissingET.Phi']))

for i in range(1,6):
    sm_tree = uproot.open(f'../../Final_Samples/delphes_ee_zh_mumutautau_sm_PDFs_50000events_230224_Run{i}.root:Delphes')
    sm_events.append(sm_tree.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 'Muon.Charge',
                     'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.TauTag', 'Jet.Charge', 'Jet.Mass',
                     'MissingET.MET', 'MissingET.Eta', 'MissingET.Phi']))
    eft_tree = uproot.open(f'../../Final_Samples/130324_chbtil_0.1_Run{i}.root:Delphes')
    eft_events.append(eft_tree.arrays(['Event.Weight','Muon_size','Muon.PT','Muon.Eta','Muon.Phi', 'Muon.Charge',
                     'Jet_size', 'Jet.PT', 'Jet.Eta', 'Jet.Phi', 'Jet.TauTag', 'Jet.Charge', 'Jet.Mass',
                     'MissingET.MET', 'MissingET.Eta', 'MissingET.Phi']))

In [3]:
"""Function to filter jet into taus only since jet size is no longer fixed to 2"""
def filter_taus(jet):
    tau_mask = []   # overall mask for all events
    for i in jet:
        tagged = [] # sub-mask for each event
        for t in i:
            tagged.append(t.tautag == 1)
        tau_mask.append(tagged)
    taus = jet[tau_mask]
    return taus

In [4]:
def ditau_mask_and_cut(events):
    jets = vector.zip({'pt':events['Jet.PT'],'eta':events['Jet.Eta'],
                   'phi':events['Jet.Phi'],'mass':events['Jet.Mass'],
                   'charge':events['Jet.Charge'],
                   'tautag':events['Jet.TauTag']})

    all_muons = vector.zip({'pt':events['Muon.PT'],'eta':events['Muon.Eta'],
                    'phi':events['Muon.Phi'],'mass':0.10566,
                    'charge':events['Muon.Charge']})

    """Check there is tau+ and tau- in the ditau system
    as well as mu+ and mu- in the dimuon system
    on top of the previous selection criteria"""
    ditau_mask = (events['Jet_size'] >= 2) & (events['Muon_size'] == 2) & (ak.sum(events['Jet.TauTag'], axis=1) == 2)
    ditau_mask = ditau_mask & (ak.sum(events['Muon.Charge'], axis=1) == 0)
    tau_charge_mask = []
    for i in events:
        tau_charge = 0
        for j,e in enumerate(i['Jet.Charge']):
            if i['Jet.TauTag'][j] == 1:
                tau_charge += e
        tau_charge_mask.append(tau_charge == 0)
    ditau_mask = ditau_mask & tau_charge_mask
    taus = filter_taus(jets[ditau_mask])

    "Apply 4 vector cuts"
    cuts = []
    muons = all_muons[ditau_mask]
    zcandidate = muons[:,0]+muons[:,1]
    recoilmass = [math.sqrt(240*240 + i.mass*i.mass - 480*i.E) for i in zcandidate]
    ditau_mass = (taus[:,0]+taus[:,1]).mass
    for n,i in enumerate(taus):
        # ditau_mass = (i[0]+i[1]).mass
        # tau_cuts.append(ak.any(i.pt < 10) or (ditau_mass > 125) or (ditau_mass < 25))
        # tau_cuts.append(ak.any(i.pt < 10))
        cut = False
        # print(muons[n])
        "Tau minimum pt cut"
        if ak.any(i.pt < 10):
            cut = True
        "Muon minimum pt cut"
        if ak.any(muons[n].pt < 10):
            cut = True
        # "Z candidate pt cut"
        # if zcandidate[n].pt < 30 or zcandidate[n].pt > 55:
        #     cut = True
        "Dimuon mass cut (for which we can use the z candidate 4 vector as it is the sum of the two muons)"
        if abs(zcandidate[n].mass - 91.2) > 10:
            cut = True
        # "Recoil mass cut"
        # if abs(recoilmass[n] - 125) > 3:
        #     cut = True
        # "Ditau mass cut"
        # if ditau_mass[n] > 125 or ditau_mass[n] < 25:
        #     cut = True
        cuts.append(cut)
    valid_events = [not i for i in cuts]
    taus = taus[valid_events]
    muons = muons[valid_events]

    ditau_events = events[ditau_mask]
    ditau_events = ditau_events[valid_events]
    weights = ditau_events['Event.Weight'] / len(events)

    return taus, muons, weights

In [8]:
"""Iterate through all SM runs, applying the ditau mask and cuts to each
And then saving the results to .npy files
set directory to the run identifier for this set of selections
which is usually the date of the run"""

run = '090324_2'

for i, events in enumerate(sm_events):
    taus, muons, weights = ditau_mask_and_cut(events)
    np.save(f'sm_data/{run}/sm_taus_run_{i+1}.npy', taus)
    np.save(f'sm_data/{run}/sm_muons_run_{i+1}.npy', muons)
    np.save(f'sm_data/{run}/sm_weights_run_{i+1}.npy', weights)

In [5]:
"""Iterate through all EFT runs, applying the ditau mask and cuts to each
And then saving the results to .npy files
set directory to the run identifier for this set of selections
which is usually the date of the run"""

run = '130324_chbtil_0.1'

for i, events in enumerate(eft_events):
    taus, muons, weights = ditau_mask_and_cut(events)
    np.save(f'eft_data/{run}/eft_taus_run_{i+1}.npy', taus)
    np.save(f'eft_data/{run}/eft_muons_run_{i+1}.npy', muons)
    np.save(f'eft_data/{run}/eft_weights_run_{i+1}.npy', weights)